In [1]:
#Import dependencies.
import pandas as pd
import numpy as np
import os
import csv
import warnings
warnings.filterwarnings('ignore')
from io import StringIO
import zipfile
from pathlib import Path

# Population Data Cleaning.

In [3]:
#Read dataframe.
conflict_df=pd.read_csv("Resources/cleaned conflict.csv",header=0)
demographics_df=pd.read_csv("Resources/demographics.csv",header=0)
disasters_df=pd.read_csv("Resources/cleaned disasters.csv",header=0)
decision_df=pd.read_csv("Resources/cleaned_asylum.csv",header=0)

In [21]:
country_list = demographics_df['Country of origin'].unique().tolist()

In [4]:
conflict_df['helper'] = conflict_df['Year'].astype(str) + conflict_df['Country of origin']
demographics_df['helper'] = demographics_df['Year'].astype(str) + demographics_df['Country of origin']
disasters_df['helper'] = disasters_df['Year'].astype(str) + disasters_df['Country of origin']
decision_df['helper'] = decision_df['Year'].astype(str) + decision_df['Country of origin']

In [107]:
# Group the data by Country of origin and Year and calculate the sum of the Total
summed_df = demographics_df.groupby(['Country of origin', 'Year', 'helper', 'Female 0 - 4', 'Female 5 - 11', 'Female 12 - 17', 'Female 18 - 59', 'Female 60', 'Female other', 'Female total', 'Male 0 - 4', 'Male 5 - 11', 'Male 12 - 17', 'Male 18 - 59', 'Male 60', 'Male other', 'Male total'], as_index=False)['Total'].sum()

# Group by Country of origin to get the full count of every year and drop if the Total count is below 50,000
filtered_df = summed_df.groupby('Country of origin').filter(lambda x: x['Total'].sum() >= 50000)

# Reset the index to ungroup the data
demo_result_df = pd.DataFrame(filtered_df.reset_index(drop=True))
demo_result_df.to_csv('Resources/demo_result_df.csv')

In [106]:
grouped_conflict_df = conflict_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_demo_result_df = demo_result_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_disasters_df = disasters_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_decision_df = decision_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_conflict_df = pd.DataFrame(grouped_conflict_df.reset_index())
grouped_demo_result_df = pd.DataFrame(grouped_demo_result_df.reset_index())
grouped_disasters_df = pd.DataFrame(grouped_disasters_df.reset_index())
grouped_decision_df = pd.DataFrame(grouped_decision_df.reset_index())
grouped_demo_result_df

,Year,Country of origin,helper,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Female total,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Male total,Total
0,2001,Afghanistan,2001Afghanistan,192133,304109,260677,581853,40530,6664,1385966,195429,322895,276762,782748,57368,8718,1643920,3809763
1,2001,Albania,2001Albania,0,5,0,10,0,436,451,0,0,0,9,0,653,662,7627
2,2001,Algeria,2001Algeria,9,0,14,23,0,304,350,10,11,10,88,0,556,675,8399
3,2001,Angola,2001Angola,12213,11425,9816,27376,2117,11626,74573,11401,12078,10357,22962,1827,14743,73368,470616
4,2001,Armenia,2001Armenia,13,22,18,68,0,257,378,11,23,21,69,5,244,373,7206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2021,Venezuela (Bolivarian Republic of),2021Venezuela (Bolivarian Republic of),373,1440,999,10870,1074,27,14783,448,1520,1045,11074,644,26,14757,199202
1833,2021,Viet Nam,2021Viet Nam,36,88,64,470,228,5,891,50,89,70,752,304,9,1274,317737
1834,2021,Western Sahara,2021Western Sahara,0,0,0,27,0,0,27,0,0,0,88,0,0,88,117041
1835,2021,Yemen,2021Yemen,741,1497,1319,5226,411,101,9295,765,1604,1464,11673,632,48,16186,37615


In [74]:
new_country_list = grouped_demo_result_df['Country of origin'].unique().tolist()
print(len(new_country_list))

88


In [75]:
dropped_countries = list(set(country_list) - set(new_country_list))
dropped_countries.sort()
print(len(dropped_countries))

113


In [103]:
grouped_disasters_df = grouped_disasters_df[~grouped_disasters_df['Country of origin'].isin(dropped_countries)]
grouped_disasters_df.head()

,Year,Country of origin,helper,Total deaths,Total affected
0,2001,Afghanistan,2001Afghanistan,331.0,200270.0
1,2001,Algeria,2001Algeria,921.0,45423.0
2,2001,Angola,2001Angola,48.0,39928.0
5,2001,Bangladesh,2001Bangladesh,232.0,530150.0
9,2001,Cambodia,2001Cambodia,56.0,1669182.0


In [12]:
grouped_conflict_df.head()

,Year,Country of origin,helper,Latitude,Longitude,Deaths civilians
0,2001,Afghanistan,2001Afghanistan,7818.191889,15174.902299,864
1,2001,Algeria,2001Algeria,2687.105591,275.155071,32
2,2001,Angola,2001Angola,-1286.696506,1834.661512,1038
3,2001,Azerbaijan,2001Azerbaijan,200.602000,231.676114,0
4,2001,Bangladesh,2001Bangladesh,343.676364,1366.693358,6


In [14]:
grouped_decision_df.head()

,Year,Country of origin,helper,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions
0,2001,Afghanistan,2001Afghanistan,18450,13729,9903,18190,60272
1,2001,Albania,2001Albania,1309,285,3957,1589,7135
2,2001,Algeria,2001Algeria,639,237,8562,2698,12105
3,2001,Angola,2001Angola,1821,1623,3143,1796,8341
4,2001,Antigua and Barbuda,2001Antigua and Barbuda,5,0,0,0,5


In [109]:

filtered_decision_df = grouped_decision_df[['Year','Country of origin','helper','Recognized decisions','Complementary protection','Rejected decisions','Otherwise closed', 'Total decisions']]
filtered_demographics_df = grouped_demo_result_df[['Year','Country of origin','helper','Female 0 - 4','Female 5 - 11','Female 12 - 17','Female 18 - 59','Female 60','Female other','Male 0 - 4','Male 5 - 11','Male 12 - 17','Male 18 - 59','Male 60','Male other','Total']]
filtered_conflict_df = grouped_conflict_df[['Year','Country of origin','helper', 'Deaths civilians']]
filtered_disasters_df = grouped_disasters_df[['Year','Country of origin','helper','Total deaths','Total affected']]
filtered_demographics_df


,Year,Country of origin,helper,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Total
0,2001,Afghanistan,2001Afghanistan,192133,304109,260677,581853,40530,6664,195429,322895,276762,782748,57368,8718,3809763
1,2001,Albania,2001Albania,0,5,0,10,0,436,0,0,0,9,0,653,7627
2,2001,Algeria,2001Algeria,9,0,14,23,0,304,10,11,10,88,0,556,8399
3,2001,Angola,2001Angola,12213,11425,9816,27376,2117,11626,11401,12078,10357,22962,1827,14743,470616
4,2001,Armenia,2001Armenia,13,22,18,68,0,257,11,23,21,69,5,244,7206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2021,Venezuela (Bolivarian Republic of),2021Venezuela (Bolivarian Republic of),373,1440,999,10870,1074,27,448,1520,1045,11074,644,26,199202
1833,2021,Viet Nam,2021Viet Nam,36,88,64,470,228,5,50,89,70,752,304,9,317737
1834,2021,Western Sahara,2021Western Sahara,0,0,0,27,0,0,0,0,0,88,0,0,117041
1835,2021,Yemen,2021Yemen,741,1497,1319,5226,411,101,765,1604,1464,11673,632,48,37615


In [9]:
# grouped_demo_df_filtered = grouped_demographics_df.loc[(grouped_demographics_df['Country of origin'] == 'Afghanistan') & (grouped_demographics_df['Year'] == 2001)]
# grouped_demo_df_filtered['Total'] = grouped_demo_df_filtered['Female total'] + grouped_demo_df_filtered['Male total']

In [13]:
grouped_disasters_df.head()

,Year,Country of origin,helper,Total deaths,Total affected
0,2001,Afghanistan,2001Afghanistan,331.0,200270.0
1,2001,Algeria,2001Algeria,921.0,45423.0
2,2001,Angola,2001Angola,48.0,39928.0
3,2001,Argentina,2001Argentina,15.0,253000.0
4,2001,Australia,2001Australia,3.0,3001.0


In [111]:
mergedf = pd.merge(filtered_demographics_df, filtered_decision_df , on=['Year', 'Country of origin','helper'], how='left')
mergedf1 = pd.merge(mergedf, filtered_conflict_df, on=['Year', 'Country of origin','helper'], how='left')
mergedf2 = pd.merge(mergedf1, filtered_disasters_df, on=['Year', 'Country of origin','helper'], how='left')
mergedf2 = mergedf2.drop('helper', axis=1)
for col in mergedf2.columns:
    if mergedf2[col].dtype == float:
        mergedf2[col] = mergedf2[col].fillna(0).astype(int)
mergedf2

,Year,Country of origin,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Male 0 - 4,Male 5 - 11,...,Male other,Total,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions,Deaths civilians,Total deaths,Total affected
0,2001,Afghanistan,192133,304109,260677,581853,40530,6664,195429,322895,...,8718,3809763,18450,13729,9903,18190,60272,864,331,200270
1,2001,Albania,0,5,0,10,0,436,0,0,...,653,7627,1309,285,3957,1589,7135,0,0,0
2,2001,Algeria,9,0,14,23,0,304,10,11,...,556,8399,639,237,8562,2698,12105,32,921,45423
3,2001,Angola,12213,11425,9816,27376,2117,11626,11401,12078,...,14743,470616,1821,1623,3143,1796,8341,1038,48,39928
4,2001,Armenia,13,22,18,68,0,257,11,23,...,244,7206,1494,252,6628,2743,11112,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2021,Venezuela (Bolivarian Republic of),373,1440,999,10870,1074,27,448,1520,...,26,199202,14907,13998,10117,18950,57972,0,0,0
1833,2021,Viet Nam,36,88,64,470,228,5,50,89,...,9,317737,863,23,3567,1240,5693,0,9,83430
1834,2021,Western Sahara,0,0,0,27,0,0,0,0,...,0,117041,69,15,605,148,837,0,0,0
1835,2021,Yemen,741,1497,1319,5226,411,101,765,1604,...,48,37615,1736,972,494,538,3740,0,13,22380


In [116]:
mergedf2['MaleFemaleTotal'] = mergedf2[['Female 0 - 4','Female 5 - 11','Female 12 - 17','Female 18 - 59','Female 60','Female other','Male 0 - 4','Male 5 - 11','Male 12 - 17','Male 18 - 59','Male 60','Male other']].sum(axis=1)
mergedf2['Unknown demographic'] = mergedf2['Total'] - mergedf2['MaleFemaleTotal']
col_to_move = mergedf2.pop('Unknown demographic')  
mergedf2.insert(19, 'Unknown demographic', col_to_move)  
mergedf2 = mergedf2.drop('MaleFemaleTotal', axis=1)
mergedf2=mergedf2.replace(to_replace="TÃ¼rkiye",
           value="Turkey")
ml_df = mergedf2
ml_df

,Year,Country of origin,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Male 0 - 4,Male 5 - 11,...,Total,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Unknown demographic,Total decisions,Deaths civilians,Total deaths,Total affected
0,2001,Afghanistan,192133,304109,260677,581853,40530,6664,195429,322895,...,3809763,18450,13729,9903,18190,779877,60272,864,331,200270
1,2001,Albania,0,5,0,10,0,436,0,0,...,7627,1309,285,3957,1589,6514,7135,0,0,0
2,2001,Algeria,9,0,14,23,0,304,10,11,...,8399,639,237,8562,2698,7374,12105,32,921,45423
3,2001,Angola,12213,11425,9816,27376,2117,11626,11401,12078,...,470616,1821,1623,3143,1796,322675,8341,1038,48,39928
4,2001,Armenia,13,22,18,68,0,257,11,23,...,7206,1494,252,6628,2743,6455,11112,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2021,Venezuela (Bolivarian Republic of),373,1440,999,10870,1074,27,448,1520,...,199202,14907,13998,10117,18950,169662,57972,0,0,0
1833,2021,Viet Nam,36,88,64,470,228,5,50,89,...,317737,863,23,3567,1240,315572,5693,0,9,83430
1834,2021,Western Sahara,0,0,0,27,0,0,0,0,...,117041,69,15,605,148,116926,837,0,0,0
1835,2021,Yemen,741,1497,1319,5226,411,101,765,1604,...,37615,1736,972,494,538,12134,3740,0,13,22380


In [118]:
ml_df.to_csv('Resources/ml_df.csv')